In [ ]:
from collections import deque
from itertools import islice

file_path = 'H:\\Flux\\BakerHall\\Baker_ts_2022_11132024.dat'

# with open(file_path, 'r') as f:

#     first_lines = list(islice(f, 10))  # 读取前5行
#     print(''.join(first_lines))

#     last_lines = deque(f, maxlen=5)  # 设置缓存最大行数为5
#     print(''.join(last_lines))

with open(file_path, 'r') as f:
    # for idx, line in enumerate(f):
    #     # 打印前 5 行
    #     if idx < 5:
    #         print(line, end='')

    #     # 每隔 1000 行打印 5 行
    #     if idx % 100000 == 0:
    #         # 从当前位置读取 5 行
    #         next_lines = list(islice(f, 10))
    #         print(f"\n--- Lines around {idx + 1} ---")
    #         print(''.join(next_lines))

    # 打开文件并读取最后 5 行
    last_lines = deque(f, maxlen=5)  # 设置 deque 的最大长度为 5
    # deque 会自动保持最新的 5 行
    

In [4]:
print(''.join(last_lines))

"2024-11-13 11:30:22",1794870101,-7.388751,-2.1865,0.2,784.7106,6.829575,9.24657,98.97175,0
"2024-11-13 11:30:22.1",1794870102,-6.50325,-2.62075,-0.2395,785.4329,6.845049,9.372296,98.9624,0
"2024-11-13 11:30:22.2",1794870103,-7.40625,-0.6765,0.37775,788.0367,6.870135,9.642369,98.95306,0
"2024-11-13 11:30:22.3",1794870104,-7.852,-1.61225,-0.99975,785.6484,6.836778,9.516531,98.97175,0
"2024-11-13 11:30:22.4",1794870105,-8.298,-1.925,0.064,785.0646,6.832483,8.216379,98.9624,0



In [1]:
from itertools import islice

file_path = 'H:\\Flux\\TOA5_ts1_240314.dat'
output_file = 'H:\\Flux\\output_subset.dat'

# 定义范围起点和行数
start_line = 0
line_count = 100000

with open(file_path, 'r') as f, open(output_file, 'w') as out_f:
    for idx, line in enumerate(f):
        # 打印前 5 行
        if idx < 5:
            print(line, end='')

        # 每隔 100,000 行打印 10 行
        # if idx % 500000 == 0:
        #     next_lines = list(islice(f, 10))
        #     print(f"\n--- Lines around {idx + 1} ---")
        #     print(''.join(next_lines))

        # 写入目标范围的行到输出文件
        if start_line <= idx < start_line + line_count:
            out_f.write(line)


"TOA5","2349","CR5000","2349","CR5000.Std.06","CPU:BIOE130626_tower1.CR5","56646","ts_data"
"TIMESTAMP","RECORD","Ux","Uy","Uz","co2","h2o","Ts","press","diag_csat","agc"
"TS","RN","m/s","m/s","m/s","mg/m^3","g/m^3","C","kPa","unitless","unitless"
"","","Smp","Smp","Smp","Smp","Smp","Smp","Smp","Smp","Smp"
"2024-02-12 13:10:24.6",5067,1.493,-2.16525,-0.61225,778.1209,4.222747,4.97246,98.01163,0,50


In [3]:
import pandas as pd
from datetime import datetime, timedelta
import csv

# 文件路径
input_file = 'G:\\flux\\organize\\-20220418\\BCK_ts2022.dat'
output_file = 'G:\\flux\\organize\\-20220418\\BCK_ts2022_corrected.dat'

# 初始化变量
header_lines = []
data_rows = []

# 读取文件并按逗号分割
with open(input_file, 'r') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i < 4:  # 前4行为表头
            header_lines.append(','.join(row))  # 保留原始表头格式
        else:
            data_rows.append(row)

# 确定时间戳列索引（假设时间戳在第1列）
timestamp_col_idx = 0  # 通常时间戳位于第0列

# 将数据部分时间戳转换为 datetime 对象
for row in data_rows:
    try:
        row[timestamp_col_idx] = datetime.strptime(row[timestamp_col_idx], '%Y-%m-%d %H:%M:%S')
    except ValueError:
        row[timestamp_col_idx] = None  # 对无法解析的时间标记为 None

# 找出年份错误的时间戳（1998年）并修复
valid_rows = [row for row in data_rows if row[timestamp_col_idx] and row[timestamp_col_idx].year > 2000]
if valid_rows:
    # 找到第一个有效时间戳和时间步长
    first_valid_time = valid_rows[0][timestamp_col_idx]
    time_step = (valid_rows[1][timestamp_col_idx] - valid_rows[0][timestamp_col_idx]).total_seconds()

    # 修复时间戳
    for row in data_rows:
        if row[timestamp_col_idx] and row[timestamp_col_idx].year == 1998:
            first_valid_time -= timedelta(seconds=time_step)
            row[timestamp_col_idx] = first_valid_time

# 将修复后的时间戳格式化为字符串
for row in data_rows:
    if row[timestamp_col_idx]:
        row[timestamp_col_idx] = row[timestamp_col_idx].strftime('%Y-%m-%d %H:%M:%S')

# 写出修复后的文件
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    # 写入表头
    for line in header_lines:
        f.write(line + '\n')
    # 写入数据部分
    writer.writerows(data_rows)

In [ ]:
from datetime import datetime
import os

file_path = 'H:\\Flux\\TOA5_ts1_240314.dat'
output_dir = 'H:\\Flux\\split_data'
os.makedirs(output_dir, exist_ok=True)

# 读取文件并按行处理
with open(file_path, 'r') as f:
    header = [next(f) for _ in range(4)]  # 读取前 4 行
    current_start_time = None
    current_file = None
    
    for line in f:
        parts = line.strip().split(',')
        timestamp_str = parts[0].strip('"')  # 去除时间字段的引号
        
        # 解析时间
        try:
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S.%f")
        except ValueError:
            continue  # 如果解析失败，跳过此行
        
        # 计算当前时间所属的 30 分钟区间
        period_start = timestamp.replace(minute=(timestamp.minute // 30) * 30, second=0, microsecond=0)
        
        if current_start_time is None:
            current_start_time = period_start
            current_file = os.path.join(output_dir, f"{current_start_time.strftime('%Y%m%d_%H%M')}.dat")
            
            # 创建新文件并写入头部
            with open(current_file, 'w') as out_f:
                out_f.writelines(header)
        
        # 检查是否需要切换到新文件
        if period_start != current_start_time:
            # 切换到新文件
            current_start_time = period_start
            current_file = os.path.join(output_dir, f"{current_start_time.strftime('%Y%m%d_%H%M')}.dat")
            
            # 创建新文件并写入头部
            with open(current_file, 'w') as out_f:
                out_f.writelines(header)
        
        # 写入当前数据行
        with open(current_file, 'a') as out_f:
            out_f.write(line)


KeyboardInterrupt: 